# ETL (Extract, Transform, Load)

Este notebook realiza o processo de ETL (Extract, Transform, Load) de dados de pedidos, consumidores e restaurantes, com o objetivo de preparar e analisar dados para experimentos de negócio. As principais etapas incluem:

- **Carregamento de Dados Brutos:** Utiliza funções utilitárias para carregar dados brutos de pedidos, consumidores, restaurantes e mapeamento de IDs.
- **Limpeza e Conformação:** Aplica regras de limpeza, tratamento de timezone e conformação dos dados para garantir consistência e qualidade.
- **Construção de Camadas Silver:** Gera tabelas intermediárias ("silver") de pedidos e usuários, agregando informações relevantes como recência, frequência e valor monetário.
- **Análises Exploratórias:** Realiza inspeções de schema, contagem de linhas, análise de nulos, faixas de datas, splits por grupos experimentais, distribuição de valores e estatísticas descritivas.
- **Validações:** Executa asserts para garantir unicidade de IDs e ausência de nulos críticos.
- **Visualizações:** Gera gráficos para explorar a distribuição dos valores de pedidos e sumariza métricas por grupo experimental.

O notebook serve como base para análises e validações de experimentos, garantindo integridade e qualidade dos dados processados.

In [1]:
# --- Bootstrap (Colab + Local) com persistência no Drive e detecção da raiz do repo ---
import os, sys, subprocess, shutil
from pathlib import Path

GITHUB_USER = "silvaniacorreia"
REPO_NAME   = "ifood-case-cupons"
REPO_URL    = f"https://github.com/{GITHUB_USER}/{REPO_NAME}.git"

IN_COLAB = "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ
PERSIST_ON_DRIVE = True 

def run(cmd):
    print(">", " ".join(cmd))
    subprocess.check_call(cmd)

def find_project_root(start: Path = None) -> Path:
    """Sobe diretórios até achar requirements.txt e src/utils.py."""
    start = start or Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / "requirements.txt").exists() and (p / "src" / "utils.py").exists():
            return p
    return start 

if IN_COLAB:
    if PERSIST_ON_DRIVE:
        from google.colab import drive
        drive.mount("/content/drive", force_remount=False)

    CONTENT_DIR = Path("/content")
    PROJECT_DIR = CONTENT_DIR / REPO_NAME
    if not PROJECT_DIR.exists():
        run(["git", "clone", REPO_URL, str(PROJECT_DIR)])
    else:
        os.chdir(PROJECT_DIR)
        run(["git", "fetch", "--all"])
        run(["git", "checkout", "main"])
        run(["git", "pull", "--rebase", "origin", "main"])
    os.chdir(PROJECT_DIR)

    if PERSIST_ON_DRIVE:
        DRIVE_PROJ = Path("/content/drive/MyDrive") / REPO_NAME
        DRIVE_DATA = DRIVE_PROJ / "data"
        DRIVE_PROJ.mkdir(parents=True, exist_ok=True)
        DRIVE_DATA.mkdir(parents=True, exist_ok=True)
        LOCAL_DATA = PROJECT_DIR / "data"
        if not LOCAL_DATA.is_symlink():
            if LOCAL_DATA.exists() and any(LOCAL_DATA.iterdir()):
                for p in LOCAL_DATA.iterdir():
                    dst = DRIVE_DATA / p.name
                    if p.is_dir():
                        shutil.copytree(p, dst, dirs_exist_ok=True)
                    else:
                        shutil.copy2(p, dst)
                shutil.rmtree(LOCAL_DATA)
            os.symlink(str(DRIVE_DATA), str(LOCAL_DATA), target_is_directory=True)

    run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    run([sys.executable, "scripts/download_data.py"])

    if str(PROJECT_DIR) not in sys.path:
        sys.path.insert(0, str(PROJECT_DIR))
    print("✔️ Bootstrap concluído (Colab). Projeto:", PROJECT_DIR)

else:
    PROJECT_DIR = find_project_root(Path.cwd().resolve())
    os.chdir(PROJECT_DIR)  
    if str(PROJECT_DIR) not in sys.path:
        sys.path.insert(0, str(PROJECT_DIR))
    print("Execução local. Raiz do projeto:", PROJECT_DIR)


Execução local. Raiz do projeto: C:\Users\silva\OneDrive\Documentos\portfólio\ifood-case-cupons


In [ ]:
from src.utils import load_settings, get_spark
from src import etl
from pyspark.sql import functions as F

s = load_settings()
spark = get_spark(
    app_name=s.runtime.spark.app_name,
    shuffle_partitions=s.runtime.spark.shuffle_partitions
)

orders, consumers, restaurants, abmap = etl.load_raw(spark, s.data.raw_dir)

win = getattr(s.analysis, "experiment_window", None)
start = win.get("start") if isinstance(win, dict) else None
end   = win.get("end")   if isinstance(win, dict) else None
auto  = getattr(s.analysis, "auto_infer_window", True)

df = etl.clean_and_conform(
    orders, consumers, restaurants, abmap,
    business_tz=getattr(s.analysis, "business_tz", "America/Sao_Paulo"),
    treat_is_target_null_as_control=getattr(s.analysis, "treat_is_target_null_as_control", False),
    experiment_start=start,
    experiment_end=end,
    auto_infer_window=auto,
)

orders_silver = etl.build_orders_silver(df)

df_test = spark.range(10)
df_test.write.mode("overwrite").parquet(os.path.abspath("data/processed/teste.parquet"))

orders_silver.write.mode("overwrite").parquet(f"{s.data.processed_dir}/orders_silver.parquet")

users_silver = etl.build_user_aggregates(orders_silver)
ref_ts = orders_silver.agg(F.max("event_ts_utc")).first()[0]
users_silver = users_silver.withColumn("recency", F.datediff(F.lit(ref_ts), F.col("last_order")))
users_silver.write.mode("overwrite").parquet(f"{s.data.processed_dir}/users_silver.parquet")

display(orders_silver.limit(5).toPandas())
display(users_silver.limit(5).toPandas())
print("ref_ts:", ref_ts)
print("ETL concluído com sucesso!")

[ETL] Janela INFERIDA a partir dos dados (UTC): start=2019-01-17 end=2019-01-18 (end exclusivo)


Py4JJavaError: An error occurred while calling o324.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(Fi

## Checagens dos dados

In [ ]:
# Esquema, contagens e unicidade de chaves

print("=== orders_silver ===")
orders_silver.printSchema()
n_orders = orders_silver.count()
n_orders_distinct = orders_silver.select("order_id").distinct().count()
print("linhas:", n_orders, "| order_id distintos:", n_orders_distinct)

print("\n=== users_silver ===")
users_silver.printSchema()
n_users  = users_silver.count()
n_users_distinct = users_silver.select("customer_id").distinct().count()
print("linhas:", n_users, "| customer_id distintos:", n_users_distinct)


In [ ]:
# Nulos por coluna 

def nulls_by_col(df):
    exprs = [F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns]
    return df.select(exprs)

print("Nulos em orders_silver:")
nulls_by_col(orders_silver).show(truncate=False)

print("Nulos em users_silver:")
nulls_by_col(users_silver).show(truncate=False)


In [ ]:
# Faixa de datas e janela do experimento

print("Faixa de datas (UTC) em orders_silver:")
orders_silver.agg(F.min("event_ts_utc").alias("min_utc"),
                  F.max("event_ts_utc").alias("max_utc")).show()

print("Contagem diária (BRT) por grupo:")
(orders_silver.groupBy("event_date_brt","is_target")
 .count()
 .orderBy("event_date_brt","is_target")
 .show(40))


In [ ]:
# Balanceamento do A/B

print("Split por is_target (orders):")
orders_silver.groupBy("is_target").count().show()

print("Split por is_target (users):")
users_silver.groupBy("is_target").count().show()


In [ ]:
# Valores monetários — sanity check e outliers

print("Negativos/zero em order_total_amount:")
neg = orders_silver.filter(F.col("order_total_amount") < 0).count()
zero = orders_silver.filter(F.col("order_total_amount") == 0).count()
print("negativos:", neg, "| zero:", zero)

print("Resumo de order_total_amount:")
orders_silver.select("order_total_amount").summary().show()

print("Quantis aproximados (1%, 5%, 50%, 95%, 99%):")
q = orders_silver.approxQuantile("order_total_amount", [0.01, 0.05, 0.5, 0.95, 0.99], 0.01)
q


In [ ]:
# Top outliers (visão pontual)

orders_silver.orderBy(F.desc("order_total_amount")).select(
    "order_id","customer_id","order_total_amount","event_ts_utc","is_target"
).show(10, truncate=False)


In [ ]:
#  Distribuições

import matplotlib.pyplot as plt

pdf = (orders_silver
       .select("order_total_amount")
       .sample(withReplacement=False, fraction=0.2, seed=42)  # 20% de amostra
       .toPandas()
       .dropna())

plt.figure()
plt.hist(pdf["order_total_amount"], bins=50)
plt.title("Distribuição de order_total_amount (amostra)")
plt.xlabel("R$")
plt.ylabel("Contagem")
plt.show()


In [ ]:
# Métricas por grupo

preview = (orders_silver.groupBy("is_target")
           .agg(
               F.count("*").alias("n_orders"),
               F.sum("order_total_amount").alias("gmv"),
               F.avg("order_total_amount").alias("avg_ticket"),
           )
          )
preview.show()


In [ ]:
# Sanidade de atributos de apoio

print("language por grupo (users):")
(users_silver.groupBy("is_target","language")
 .count()
 .orderBy("language","is_target")
 .show(20))

print("active rate por grupo (users):")
(users_silver.groupBy("is_target")
 .agg(F.avg(F.col("recency").isNotNull().cast("double")).alias("has_recency"),
      F.avg(F.col("frequency")).alias("avg_freq"),
      F.avg(F.col("monetary")).alias("avg_monetary"))
 .show())


In [ ]:
# Checklist final do ETL

assert n_orders == n_orders_distinct, "order_id duplicado no silver de pedidos"
assert n_users == n_users_distinct,   "customer_id duplicado no silver de usuários"
assert orders_silver.filter(F.col("event_ts_utc").isNull()).count() == 0, "event_ts_utc nulo"
assert users_silver.filter(F.col("is_target").isNull()).count() == 0, "is_target nulo em users"
print("✔️ ETL checks básicos OK.")
